In [6]:
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
import time
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [7]:
cursos = ['15 Secrets Successful People Know About Time Management (getAbstract Summary)',
'360-Degree Feedback',
'AWS API Gateway with HTTP, Lambda, DynamoDB, and iOS',
'Accelerating TensorFlow with the Google Machine Learning Engine',
'Accounting Foundations: Asset Impairment',
'Accounting Foundations: Budgeting',
'Advanced SEO: Developing an SEO-Friendly Website',
'Agile Marketing Foundations',
'Agile at Work: Planning with Agile User Stories',
'Agilidade de Aprendizagem',
'Amazon Web Services - Você está entre os melhores 70%',
'Balancing Innovation and Risk',
'Become a Purpose-Driven Sales Professional',
'Being a Good Mentor',
'Body Language for Leaders',
'Branding for Designers',
'Breakeven and Cost-Volume-Profit (CVP) Analysis',
'Breaking Out of a Rut',
'Building Creative Organizations',
'Building Self-Confidence',
'Building an Integrated Online Marketing Plan',
'Building and Maintaining Your UX Design Portfolio',
'Burnout: Como Evitar o Esgotamento no Trabalho',
'Business Analytics Foundations: Predictive, Prescriptive, and Experimental Analytics',
'Business Innovation Foundations',
'Coaching Employees through Difficult Situations',
'Como Aproveitar ao Máximo o LinkedIn Learning',
'Como Criar Estratégias de Crescimento',
'Como Cultivar uma Mentalidade de Crescimento',
'Como Desenvolver a Resiliência',
'Como Desenvolver sua Inteligência Emocional',
'Como Fazer Perguntas Poderosas para Vendas',
'Como Liderar e Trabalhar em Equipe',
'Como Melhorar a Prospecção de Vendas',
'Como Melhorar suas Habilidades de Escuta',]

In [8]:
df = pd.DataFrame(columns = ["url_cursos"])
driver = webdriver.Chrome('chromedriver.exe')
for j,curso in enumerate(cursos):
    driver.get('https://www.google.com.br')
    time.sleep(3)
    search_input = driver.find_element_by_name('q')
    # let google find any linkedin user with keyword "python developer" and "San Francisco"
    search_input.send_keys( curso + '" AND "Linkedin"')
    search_input.send_keys(Keys.RETURN)
    time.sleep(5)
    url_cursos = driver.find_element_by_id("search").find_element_by_css_selector('a').get_attribute('href')
    print(url_cursos)
    df = df.append(pd.DataFrame([[url_cursos]],columns = df.columns))

https://www.linkedin.com/learning/15-secrets-successful-people-know-about-time-management-getabstract-summary
https://www.linkedin.com/learning/360-degree-feedback
https://www.linkedin.com/learning/aws-api-gateway-with-http-lambda-dynamodb-and-ios
https://www.linkedin.com/learning/accelerating-tensorflow-with-the-google-machine-learning-engine
https://www.linkedin.com/learning/accounting-foundations-asset-impairment
https://www.linkedin.com/learning/accounting-foundations-budgeting-2
https://www.linkedin.com/learning/advanced-seo-developing-an-seo-friendly-website
https://www.linkedin.com/learning/agile-marketing-foundations
https://www.linkedin.com/learning/agile-at-work-planning-with-agile-user-stories
https://br.linkedin.com/learning/agilidade-de-aprendizagem
https://br.linkedin.com/company/datarainconsulting
http://scholar.google.com.br/scholar?q=Balancing+Innovation+and+Risk%22+AND+%22Linkedin%22&hl=pt-BR&as_sdt=0&as_vis=1&oi=scholart
https://www.linkedin.com/learning/purpose-driv

In [13]:
df = df[((df["url_cursos"].str.contains("linkedin.com/learning", na = False)))]

In [15]:
df_learning = pd.DataFrame(columns = ["formato","nome", "instituicao", "site_plataforma", "conteudo","professor","descricao","nivel", "rating", "skills", "publico", "data", "duracao", "preco","periodo", "idioma", "modalidade", "img"])
driver = webdriver.Chrome('chromedriver.exe')
for url_curso in df["url_cursos"]:
    driver.get(url_curso)
    seconds_per_unit = {"s": 1, "m": 60, "h": 3600, "d": 86400, "w": 604800}
    def convert_to_seconds(t):
        t = t.split(" ")
        s = 0
        for e in t:
            s = s + int(e[:-1]) * seconds_per_unit[e[-1]]
        return s
    div_conteudos = driver.find_element_by_class_name("table-of-contents__chapters").find_elements_by_class_name("chapter__label-more")
    conteudo = "\n".join([conteudo.text for conteudo in div_conteudos])
    cliques = driver.find_element_by_class_name("table-of-contents__chapters").find_elements_by_class_name("show-more-less-state__label-more")
    for clique in cliques: 
        clique.click()
    formato = "Curso Livre Online"
    nome = driver.find_element_by_class_name("top-card-layout__entity-info").find_element_by_css_selector("h1").text
    instituicao = "Linkedin Learning"
    site_plataforma = "https://www.linkedin.com/learning/me"
    professor = driver.find_element_by_class_name("instructors__list").find_element_by_css_selector("h3").text
    descricao = driver.find_element_by_class_name("course-details").find_element_by_class_name("section-container__description").text
    nivel = driver.find_element_by_class_name("top-card-layout__entity-info").find_elements_by_class_name("top-card__metadata")[1].text.replace("Iniciante","Básico").replace("+", "/").replace("Geral","-")
    rating = "-"
    skills =""
    publico = "-"
    data ="-"
    duracao_aulas = driver.find_element_by_class_name("table-of-contents__chapters").find_elements_by_class_name("video__length")
    duracoes = []
    for duracao_aula in duracao_aulas:
        duracoes.extend([duracao_aula.text])
    duracao = str(pd.DataFrame(duracoes)[0].apply(lambda x : convert_to_seconds(x)).sum()//60) + "min"
    driver.find_element_by_class_name_name("upsell-cta-purchase-form__submit")
    try:
        preco = "R$" + driver.find_element_by_class_name("buy-course-upsell__button").text.split("BRL")[-1].replace("*)","")
    except:
        preco = "-"
    periodo = "-"
    idioma = "Português"
    modalidade = "Online"
    img = "LinkedinLearning"
    #path = 'G:/Meu Drive/BRG - SkillTrack/Design/Imagens das ofertas/np'
    #img_name = path + "NP_" + str(index_artigo) + '.jpg'
    #img_data = requests.get(img).content
    #with open(img_name, 'wb') as handler:
        #handler.write(img_data)
    df_learning = df_learning.append(pd.DataFrame([[formato, nome, instituicao, site_plataforma, conteudo, professor, descricao, nivel, rating, skills, publico, data, duracao, preco, periodo, idioma, modalidade, img]],columns = df_learning.columns))

In [16]:
df_learning

,formato,nome,instituicao,site_plataforma,conteudo,professor,descricao,nivel,rating,skills,publico,data,duracao,preco,periodo,idioma,modalidade,img
0,Curso Livre Online,15 Secrets Successful People Know about Time M...,Linkedin Learning,https://www.linkedin.com/learning/me,1. 15 Secrets Successful People Know About Tim...,getAbstract,"In his book, 15 Secrets Successful People Know...",Básico,-,,-,-,8min,-,-,Português,Online,LinkedinLearning
0,Curso Livre Online,360-Degree Feedback,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. Understanding 360 Feedback\n2...,Sara Canaday,An organization's talent is vital to its succe...,Intermediário,-,,-,-,31min,"R$69,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,"AWS API Gateway with HTTP, Lambda, DynamoDB, a...",Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. Getting Started with API Gate...,Bear Cahill,Learn how to create RESTful APIs the easy way ...,Intermediário,-,,-,-,93min,"R$89,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Accelerating TensorFlow with the Google Machin...,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. Introducing TensorFlow\n2. Fu...,Matt Scarpino,Discover how to leverage TensorFlow—an open-so...,Intermediário,-,,-,-,185min,"R$79,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Accounting Foundations: Asset Impairment,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. Accounting for the Acquisitio...,Jim Stice,"Goodwill, accounts receivable, and other long-...",Avançado,-,,-,-,89min,"R$99,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Accounting Foundations: Budgeting,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. The Budgeting Process in Orga...,Jim Stice,Running an organization without a proper budge...,Básico,-,,-,-,97min,"R$69,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Advanced SEO: Developing an SEO-Friendly Website,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. Optimized Website Architectur...,Brad Batesole,Advanced SEO goes beyond keyword research and ...,Avançado,-,,-,-,158min,"R$99,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Agile Marketing Foundations,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. The Agile Marketing Playbook\...,Chris DallaVilla,"By adopting agile marketing principles, you ca...",Básico / Intermediário,-,,-,-,73min,"R$59,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Agile at Work: Planning with Agile User Stories,Linkedin Learning,https://www.linkedin.com/learning/me,Introduction\n1. Understanding Agile Planning\...,Doug Rose,Agile project teams create short user stories ...,Intermediário,-,,-,-,51min,"R$79,99",-,Português,Online,LinkedinLearning
0,Curso Livre Online,Agilidade de Aprendizagem,Linkedin Learning,https://www.linkedin.com/learning/me,Introdução\n1. Como fazer brainstorming de sua...,Gary Bolles,Você está bem encaminhado com suas metas de ap...,-,-,,-,-,36min,-,-,Português,Online,LinkedinLearning


In [72]:
driver = webdriver.Chrome('chromedriver.exe')
url_artigo = "https://www.linkedin.com/learning/pensamento-estrategico"
driver.get(url_artigo)
time.sleep(10)
driver.find_element_by_class_name("table-of-contents__chapters").find_element_by_class_name("show-more-less-state__label-more").click()
driver.find_element_by_class_name("table-of-contents__chapters").find_element_by_class_name("video__length").text

'1m 18s'

In [187]:
df_NP.merge(df, how = "left", on = "url_artigo").to_excel("Artigos_np.xlsx")